# Data Analytics Assignment

## 1. Environment Setup & Data Loading

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')

file_path = 'FJ Assignment - FJ Assignment - Sheet1(1).csv'
try:
    df = pd.read_csv(file_path)
    print(f"Loaded dataset with {df.shape[0]} rows and {df.shape[1]} columns")
    display(df.head())
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found. Please check the path.")

: 

## 2. Data Cleaning

- **Charge**: Remove '$', ',' and convert to float.
- **Weight**: Remove 'lbs' and convert to float.
- **Zones**: Standardize to numeric (remove 'Zone ').
- **Date**: Convert to datetime.

In [ ]:
# Clean 'Charge' column
if df['Charge'].dtype == 'object':
    df['Charge'] = df['Charge'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
    df['Charge'] = pd.to_numeric(df['Charge'], errors='coerce')

# Clean 'Weight (lbs)' column
if df['Weight (lbs)'].dtype == 'object':
    df['Weight (lbs)'] = df['Weight (lbs)'].astype(str).str.lower().str.replace('lbs', '', regex=False).str.strip()
    df['Weight (lbs)'] = pd.to_numeric(df['Weight (lbs)'], errors='coerce')

# Clean 'Zones' column
df['Zones'] = df['Zones'].astype(str).str.lower().str.replace('zone', '', regex=False).str.strip()
df['Zones'] = pd.to_numeric(df['Zones'], errors='coerce')

# Clean Date, handling errors
df['Date of Delivery'] = pd.to_datetime(df['Date of Delivery'], errors='coerce')

print("Data Types after cleaning:")
print(df.dtypes)

# Date Analysis
cutoff_date = pd.Timestamp('2025-01-01')
df['Dataset_Type'] = df['Date of Delivery'].apply(lambda x: 'Future/Simulated' if x > cutoff_date else 'Historical')

print("\n--- Dataset Segmentation ---")
print(df['Dataset_Type'].value_counts())

## 3. Exploratory Data Analysis (EDA)

### 3.1 Charge Types Distribution

In [ ]:
plt.figure(figsize=(12, 6))
df['Charge Type'].value_counts().head(20).plot(kind='barh')
plt.title('Top 20 Charge Types')
plt.xlabel('Count')
plt.gca().invert_yaxis()
plt.show()

## 4. Cost Modeling & Aggregation

### 4.1 Categorization Logic
- **Base**: Core shipping cost ('Base Rate', 'Freight').
- **Surcharge**: Mandatory fees (Fuel, Delivery Area, Residential).
- **Penalty/Adjustment**: Avoidable costs or corrections (Address Correction, Late, Return).
- **Other**: Anything else.

In [ ]:
def classify_charge(charge_type):
    ct_lower = str(charge_type).lower()
    
    # Base
    if ct_lower in ['base rate', 'freight']:
        return 'Base'
    
    # Penalties / Adjustments
    if any(x in ct_lower for x in ['correction', 'adjustment', 'penalty', 'late', 'return']):
        return 'Penalty/Adjustment'
    
    # Surcharges (Default bucket for non-base, non-penalty)
    return 'Surcharge'

df['Category'] = df['Charge Type'].apply(classify_charge)

# Check distribution
print(df['Category'].value_counts())

### 4.2 Aggregation per Shipment
We aggregate grouped by `Tracking Number` to get total shipment cost. We also retain `Dataset_Type` to compare Historical vs Future data.

In [ ]:
# Aggregating
aggregated = df.groupby(['Tracking Number', 'Carrier Name', 'Zones', 'Dataset_Type']).apply(
    lambda x: pd.Series({
        'Total_Cost': x['Charge'].sum(),
        'Base_Cost': x[x['Category'] == 'Base']['Charge'].sum(),
        'Surcharge_Cost': x[x['Category'] == 'Surcharge']['Charge'].sum(),
        'Penalty_Cost': x[x['Category'] == 'Penalty/Adjustment']['Charge'].sum(),
        'Num_Charges': len(x)
    })
).reset_index()

aggregated['Has_Base_Rate'] = aggregated['Base_Cost'] > 0

print("Shipment Summary Head:")
display(aggregated.head())

# Check for Missing Base Rates in Historical Data
historical_shipments = aggregated[aggregated['Dataset_Type'] == 'Historical']
missing_base = historical_shipments[~historical_shipments['Has_Base_Rate']]
print(f"\nHistorical Shipments: {len(historical_shipments)}")
print(f"Historical Shipments missing Base Rate: {len(missing_base)}")
if len(missing_base) > 0:
    print("Example Missing Base Rate:", missing_base['Tracking Number'].iloc[0])

### 4.3 Normalized Comparison (Historical Data Only)
For fair comparison, we focus on Historical Data where actual shipping patterns are observed. Future data appears to be simulated/uniform.

In [ ]:
plt.figure(figsize=(14, 6))
sns.boxplot(x='Carrier Name', y='Total_Cost', hue='Zones', data=historical_shipments)
plt.title('Historical Total Cost by Carrier and Zone')
plt.legend(title='Zone', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.show()

## 5. "Worst 10%" Analysis

We identify the 90th percentile of cost. We check if this is driven by specific carriers or charge types.

In [ ]:
# Using ALL data for Worst 10% as requested (to catch potential future high costs)
threshold_90 = aggregated['Total_Cost'].quantile(0.90)
worst_10_df = aggregated[aggregated['Total_Cost'] >= threshold_90]

print(f"90th Percentile Threshold: ${threshold_90:.2f}")
print(f"Number of Shipments in Worst 10%: {len(worst_10_df)}")

# Breakdown by Type
print("\nDataset Composition of Worst 10%:")
print(worst_10_df['Dataset_Type'].value_counts())

# Drivers Analysis - Carrier
plt.figure(figsize=(12, 6))
sns.countplot(y='Carrier Name', data=worst_10_df, order=worst_10_df['Carrier Name'].value_counts().index)
plt.title('Count of "Worst 10%" Shipments by Carrier')
plt.show()

# Drivers Analysis - Zone
plt.figure(figsize=(12, 6))
sns.countplot(x='Zones', data=worst_10_df)
plt.title('Worst 10% by Zone')
plt.show()

## 6. Data Export

Exporting the aggregated and cleaned dataset for external use.

In [ ]:
output_file = 'processed_shipment_data.csv'
aggregated.to_csv(output_file, index=False)
print(f"Processed data saved to {output_file}")